In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
sales = pd.DataFrame({
    'id': [101, 102, 103, 104, 105],
    'client_id': [1, 2, 1, 3, 3],
    'item': ['TechBook Pro 15', 'ViewMax 24HD', 'KeyPro MX', 'TechBook Air', 'ClickOne Mini'],
    'price': [1200, 999, 599, 1500, 99]
})

clients = pd.DataFrame({
    'client_id': [1, 2, 3, 4, 5],
    'client_name': ['John', 'Sergio', 'Michael', 'Eve', 'Dorian'],
    'city_id': [10, 11, 10, 11, 10]
})

cities = pd.DataFrame({
    'city_id': [10, 11],
    'city_name': ['Chicago', 'Washington']
})

In [3]:
df = sales.merge(clients, on = 'client_id', how = 'outer')
df = df.merge(cities, on = 'city_id', how = 'outer')
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,client_id,item,price,client_name,city_id,city_name
0,101.0,1,TechBook Pro 15,1200.0,John,10,Chicago
1,103.0,1,KeyPro MX,599.0,John,10,Chicago
2,104.0,3,TechBook Air,1500.0,Michael,10,Chicago
3,105.0,3,ClickOne Mini,99.0,Michael,10,Chicago
4,NaN,5,NaN,NaN,Dorian,10,Chicago
5,102.0,2,ViewMax 24HD,999.0,Sergio,11,Washington
6,NaN,4,NaN,NaN,Eve,11,Washington


In [4]:
show = df[['client_name', 'item', 'price']]
show[~show['item'].isna()]

,client_name,item,price
0,John,TechBook Pro 15,1200.0
1,John,KeyPro MX,599.0
2,Michael,TechBook Air,1500.0
3,Michael,ClickOne Mini,99.0
5,Sergio,ViewMax 24HD,999.0


In [5]:
#show clients who didn't place order:
print(f"Clients who didn't place order are: {df.loc[df['item'].isna(), 'client_name'].tolist()}")

Clients who didn't place order are: ['Dorian', 'Eve']


In [6]:
#show client, item, city:
s = df[['client_name', 'item', 'city_name']]
s[~s['item'].isna()]

,client_name,item,city_name
0,John,TechBook Pro 15,Chicago
1,John,KeyPro MX,Chicago
2,Michael,TechBook Air,Chicago
3,Michael,ClickOne Mini,Chicago
5,Sergio,ViewMax 24HD,Washington


In [7]:
#show what city has more orders:
#let's replace nans in items with "No order":
df['item'] = df['item'].fillna('No order')
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,client_id,item,price,client_name,city_id,city_name
0,101.0,1,TechBook Pro 15,1200.0,John,10,Chicago
1,103.0,1,KeyPro MX,599.0,John,10,Chicago
2,104.0,3,TechBook Air,1500.0,Michael,10,Chicago
3,105.0,3,ClickOne Mini,99.0,Michael,10,Chicago
4,NaN,5,No order,NaN,Dorian,10,Chicago
5,102.0,2,ViewMax 24HD,999.0,Sergio,11,Washington
6,NaN,4,No order,NaN,Eve,11,Washington


In [8]:
#create a dict where key is city_name, value is item and print the dict to make sure it worked:
city_dict = {}
city = df.groupby('city_name')['item'].apply(lambda x: (x != 'No order').sum())
for key, value in city.items():
    city_dict[key] = value
city_dict

{'Chicago': 4, 'Washington': 1}

In [9]:
#the dict is created, now get the key that has the highest value
max_city = max(city_dict, key = city_dict.get)
print(f"The city with the most orders is: {max_city}")

The city with the most orders is: Chicago


In [10]:
#what client placed more orders:
#the same approach: create a dict where key is a client, value is the number of order. Print the key with the highest value:
dict_clients = {}
client = df.groupby('client_name')['item'].apply(lambda x: (x != 'No order').sum())
for k, v in client.items():
    dict_clients[k] = v
dict_clients

{'Dorian': 0, 'Eve': 0, 'John': 2, 'Michael': 2, 'Sergio': 1}

And here, I realized that the same approach wouldn't work. There are several clients, so I'll have to create a dict, find the highest value, then initiate a list and append it with the keys linked to the highest value.

In [11]:
highest_sales = max(dict_clients.values())
max_clients = []
for key, value in dict_clients.items():
    if value == highest_sales:
        max_clients.append(key)
if len(max_clients) > 1:
    print(f"Clients who placed more orders are: {max_clients}.")
else:
    print(f"The client who placed more orders is: {max_clients}.")

Clients who placed more orders are: ['John', 'Michael'].
